In [1]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
import pandas as pd
import string
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [4]:
data = pd.read_csv('train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
data['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [7]:
target_labels =pd.Index(['toxic','severe_toxic','obscene','threat','insult','identity_hate'])

In [8]:
data.toxic.value_counts(normalize=True)

0    0.904156
1    0.095844
Name: toxic, dtype: float64

In [9]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
data_count = data.iloc[0:,2:].sum()

In [11]:
data_count

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [12]:
import re
import string

In [13]:
alphanum = lambda x : re.sub('\w*\d\w*','',x)
rempunc =lambda x : re.sub('[%s]'%re.escape(string.punctuation),'',x.lower())
remln = lambda x : re.sub('\n','',x)
newAssic =lambda x : re.sub(r'[^\x00-\x7f]',r'',x)
data['comment_text'] = data['comment_text'].map(alphanum).map(rempunc).map(remln).map(newAssic)
data['comment_text'][0]

'explanationwhy the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now'

In [14]:
data_tox =data.loc[:,['id','comment_text','toxic']]

In [15]:
data_tox

,id,comment_text,toxic
0,0000997932d777bf,explanationwhy the edits made under my usernam...,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0
3,0001b41b1c6bb37e,morei cant make any real suggestions on improv...,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0
...,...,...,...
159566,ffe987279560d7ff,and for the second time of asking when your vi...,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0
159568,ffee36eab5c267c9,spitzer umm theres no actual article for prost...,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0


In [16]:
data_ser =data.loc[:,['id','comment_text','severe_toxic']]

In [17]:
data_obs =data.loc[:,['id','comment_text','obscene']]

In [18]:
data_thr =data.loc[:,['id','comment_text','threat']]

In [19]:
data_ins =data.loc[:,['id','comment_text','insult']]

In [20]:
data_ide =data.loc[:,['id','comment_text','identity_hate']]

In [21]:
data_tox.head()

,id,comment_text,toxic
0,0000997932d777bf,explanationwhy the edits made under my usernam...,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0
3,0001b41b1c6bb37e,morei cant make any real suggestions on improv...,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0


In [22]:
data_tox_1 =data_tox[data_tox['toxic'] == 1].iloc[0:5000,:]

In [23]:
data_tox_1.shape

(5000, 3)

In [24]:
data_tox_0=data_tox[data_tox['toxic'] == 0].iloc[0:5000,:]

In [25]:
data_tox_done =pd.concat([data_tox_1,data_tox_0],axis=0)

In [26]:
data_tox_done

,id,comment_text,toxic
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1
12,0005c987bdfc9d4b,hey what is it talk what is it an exclusive g...,1
16,0007e25b2121310b,bye dont look come or think of comming back to...,1
42,001810bf8c45bf5f,you are gay or antisemmitian archangel white t...,1
43,00190820581d90ce,fuck your filthy mother in the ass dry,1
...,...,...,...
5544,0ece9249484eddb9,electronic intifada is not an activist site wh...,0
5545,0ece971b58a3def7,new genre the new genre supernatural was used ...,0
5547,0ecf7cde3391615b,translated as the second vatican council a un...,0
5549,0ed11396420e8531,purposethis rfc will be used to determine comm...,0


In [27]:
data_ser_1 =data_ser[data_ser['severe_toxic'] == 1].iloc[0:1595,:]
data_ser_0 =data_ser[data_ser['severe_toxic'] == 0].iloc[0:1595,:]
data_ser_done =pd.concat([data_ser_1,data_ser_0],axis=0)
data_ser_done


,id,comment_text,severe_toxic
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1
55,0020e7119b96eeeb,stupid peace of shit stop deleting my stuff as...,1
181,006e87872c8b370c,you are a stupid fuck and your mothers cunt st...,1
442,01208d2b76624130,hi im a fucking bitch,1
579,018663f910e0bfe6,what a motherfucking piece of crap those fuckh...,1
...,...,...,...
1607,044df95de9cc9daa,can you offer advice on how to get the kanji,0
1608,044f008b0f522099,re from coupon to groupon please dont post or...,0
1609,044f39de7fce756e,you have no friends in real life what makes yo...,0
1610,044f7f9bbf278968,there are plenty of video game articles that n...,0


In [28]:
data_obs_1 =data_obs[data_obs['obscene'] == 1].iloc[0:5000,:]
data_obs_0 =data_obs[data_obs['obscene'] == 0].iloc[0:5000,:]
data_obs_done =pd.concat([data_obs_1,data_obs_0],axis=0)
data_obs_done.shape


(10000, 3)

In [29]:
data_thr_1 =data_thr[data_thr['threat'] == 1].iloc[0:478,:]
data_thr_0 =data_thr[data_thr['threat'] == 0].iloc[0:1912,:]
data_thr_done =pd.concat([data_thr_1,data_thr_0],axis=0)
data_thr_done.shape


(2390, 3)

In [30]:
data_ins_1 =data_ins[data_ins['insult'] == 1].iloc[0:5000,:]
data_ins_0 =data_ins[data_ins['insult'] == 0].iloc[0:5000,:]
data_ins_done =pd.concat([data_ins_1,data_ins_0],axis=0)
data_ins_done.shape


(10000, 3)

In [31]:
data_ide_1 =data_ide[data_ide['identity_hate'] == 1].iloc[0:1450,:]
data_ide_0 =data_ide[data_ide['identity_hate'] == 0].iloc[0:5620,:]
data_ide_done =pd.concat([data_ide_1,data_ide_0],axis=0)
data_ide_done.shape


(7025, 3)

In [32]:
from sklearn import preprocessing 
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import f1_score,precision_score,recall_score,precision_recall_curve,fbeta_score,confusion_matrix,accuracy_score
from sklearn.metrics import roc_auc_score , roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.svm import  LinearSVC
from sklearn.ensemble import  RandomForestClassifier
import pandas as pd


In [33]:
df1 = pd.read_csv('train.csv')
X= df1['comment_text']
y= df1['toxic']
x_train,x_test,y_train,y_test = train_test_split(X,y ,test_size=0.2,random_state=100 )

In [34]:
x_test

42252                                 Hammed it is, cheers!
4197      Not a problem, sorry for the inconvenience and...
94042     Sources for Gambia at the 2000 Summer Olympics...
43020     Added some criticism \n\nI added some much nee...
92647     Who cares? It's just a song, it's a great song...
                                ...                        
131572                                Who's ur Uncle, Dick?
13532     "\n\nExpo Line edits\nWelcome to Wikipedia! Th...
16697     Unless you can find a direct link to the games...
39230                                  Well, cool!   (Talk)
127354    "\n\n What Does She Acctually Own??? \n\nCos, ...
Name: comment_text, Length: 31915, dtype: object

In [35]:
def train_test(df_done,labels,vectorizer,ngram):
    X= df_done.comment_text
    y = df_done[labels]
    x_train,x_test,y_train,y_test = train_test_split(X,y ,test_size=0.2,random_state=100 )
    cv = vectorizer(ngram_range=(ngram),stop_words='english')
    X_train_cv= cv.fit_transform(x_train)
    X_test_cv = cv.transform(x_test)
    
    logreg= LogisticRegression()
    logreg.fit(X_train_cv,y_train)
    print("LogisticRegression")
    
    
    knn=KNeighborsClassifier()
    knn.fit(X_train_cv,y_train)
    print("KNeighborsClassifier")
    
    bnb = BernoulliNB()
    bnb.fit(X_train_cv, y_train)
    print('BernoulliNB')
    
    mnb = MultinomialNB()
    mnb.fit(X_train_cv,y_train)
    print('MultinomialNB')
    
    
    svm_model =LinearSVC()
    svm_model.fit(X_train_cv,y_train)
    print("LinearSVC()")
    
    randomforest = RandomForestClassifier(n_estimators=100,random_state=1000)
    randomforest.fit(X_train_cv,y_train)
    print("RandomForestClassifier")
    
    f1_score_data ={'F1 Score':[f1_score(logreg.predict(X_test_cv),y_test),f1_score(knn.predict(X_test_cv),y_test),
                                f1_score(bnb.predict(X_test_cv),y_test),f1_score(mnb.predict(X_test_cv),y_test),
                               f1_score(svm_model.predict(X_test_cv),y_test),f1_score(randomforest.predict(X_test_cv),y_test)],
                    'Accuracy':[accuracy_score(logreg.predict(X_test_cv),y_test),accuracy_score(knn.predict(X_test_cv),y_test),
                               accuracy_score(bnb.predict(X_test_cv),y_test),accuracy_score(mnb.predict(X_test_cv),y_test),
                               accuracy_score(svm_model.predict(X_test_cv),y_test),accuracy_score(randomforest.predict(X_test_cv),y_test)]}
    
    df_f1 = pd.DataFrame(f1_score_data,index=['LogRegration','KNN','BernoulliNB','MultinomialNB','svm','RandomForestClassifier'])
    return df_f1

In [36]:
df_tox_cv = train_test(data_tox_done,'toxic',TfidfVectorizer,(1,1))
df_tox_cv.rename(columns ={'F1 Score':'F1 Score(toxic)'},inplace=True)
df_tox_cv.rename(columns ={'Accuracy':'Accuracy(toxic)'},inplace=True)

F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


In [37]:
df_tox_cv

,F1 Score(toxic),Accuracy(toxic)
LogRegration,0.866494,0.8715
KNN,0.256499,0.5685
BernoulliNB,0.761359,0.7085
MultinomialNB,0.870051,0.8720
svm,0.871979,0.8755
RandomForestClassifier,0.835363,0.8445


In [38]:
df_ser_cv = train_test(data_ser_done,'severe_toxic',TfidfVectorizer,(1,1))
df_ser_cv.rename(columns ={'F1 Score':'F1 Score(severe_toxic)'},inplace=True)
df_ser_cv.rename(columns ={'Accuracy':'Accuracy(severe_toxic)'},inplace=True)


df_obs_cv = train_test(data_obs_done,'obscene',TfidfVectorizer,(1,1))
df_obs_cv.rename(columns ={'F1 Score':'F1 Score(obscene)'},inplace=True)
df_obs_cv.rename(columns ={'Accuracy':'Accuracy(obscene)'},inplace=True)



df_thr_cv = train_test(data_thr_done,'threat',TfidfVectorizer,(1,1))
df_thr_cv.rename(columns ={'F1 Score':'F1 Score(threat)'},inplace=True)
df_thr_cv.rename(columns ={'Accuracy':'Accuracy(threat)'},inplace=True)




df_ins_cv = train_test(data_ins_done,'insult',TfidfVectorizer,(1,1))
df_ins_cv.rename(columns ={'F1 Score':'F1 Score(insult)'},inplace=True)
df_ins_cv.rename(columns ={'Accuracy':'Accuracy(insult)'},inplace=True)




df_ide_cv = train_test(data_ide_done,'identity_hate',TfidfVectorizer,(1,1))
df_ide_cv.rename(columns ={'F1 Score':'F1 Score(identity_hate)'},inplace=True)
df_ide_cv.rename(columns ={'Accuracy':'Accuracy(identity_hate)'},inplace=True)









F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
KNeighborsClassifier
BernoulliNB
MultinomialNB
LinearSVC()
RandomForestClassifier


In [39]:
import pickle

In [40]:
def pickle_model(df, label):
    
    X = df.comment_text
    y = df[label]

    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_vect = tfv.fit_transform(X)  
    
    with open(r"{}.pkl".format(label + '_vect'), "wb") as f:   
        pickle.dump(tfv, f)   
        
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_vect, y)

    with open(r"{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(randomforest, f)

In [41]:
datalist = [data_tox_done, data_ser_done, data_obs_done, data_ins_done, data_thr_done, data_ide_done]
label = ['toxic', 'severe_toxic', 'obscene', 'insult', 'threat', 'identity_hate']

for i,j in zip(datalist,label):
    pickle_model(i, j)